<a href="https://colab.research.google.com/github/nikshrimali/TSAI_END/blob/main/Capstone/PythonCodeGen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/nikshrimali/TSAI_END

Cloning into 'TSAI_END'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 125 (delta 42), reused 73 (delta 20), pack-reused 0
Receiving objects: 100% (125/125), 37.06 MiB | 38.88 MiB/s, done.
Resolving deltas: 100% (42/42), done.


In [2]:
# Importing necessary Libraries

from TSAI_END.Capstone.model import Encoder, Decoder, Seq2Seq
from TSAI_END.Capstone.train import train
from TSAI_END.Capstone.evaluate import evaluate
import torch
import time, math

import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s : %(levelname)s : %(message)s')
logger = logging.getLogger('PythonCodeGen')
from TSAI_END.Capstone.utils import load_config, epoch_time


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
import torch.nn as nn

print(f'Currently running on {device}')


Currently running on cuda


In [3]:
# Config

%%writefile /content/TSAI_END/Capstone/pycodechatbot.yml
name: PyCodeGen
save_dir: saved/
seed: 1
target_device: 0

modelpath:
    type: PathModel
    args: savedmodel.h5    
      

train_iterator:
    type: Train_Iterator
    args:
        batch_size: 5
        num_workers: 4
        shuffle: True


criterion: cross_entropy_loss

optimizer:
    type: SGD
    args:
        lr: 0.0005
        momentum: 0.95
        weight_decay: 0.000003


data:
    file_name: TSAI_END/Capstone/data/english_python_dataset.txt

epochs: 30

Overwriting /content/TSAI_END/Capstone/pycodechatbot.yml


In [4]:
config = load_config('/content/TSAI_END/Capstone/pycodechatbot.yml')


BATCH_SIZE = config['train_iterator']['args']['batch_size']
FILEPATH = config['data']['file_name']
LEARNING_RATE = config['optimizer']['args']['lr']
N_EPOCHS = config['epochs']

In [5]:
from TSAI_END.Capstone.data import GetData
data = GetData(FILEPATH)
STAT, CODE, train_iterator, valid_iterator = data.load_iterator(batch_size=BATCH_SIZE, device=device)

In [6]:


INPUT_DIM = len(STAT.vocab)
OUTPUT_DIM = len(CODE.vocab)
HID_DIM = 256
ENC_LAYERS = 3
DEC_LAYERS = 3
ENC_HEADS = 8
DEC_HEADS = 8
ENC_PF_DIM = 512
DEC_PF_DIM = 512
ENC_DROPOUT = 0.1
DEC_DROPOUT = 0.1

enc = Encoder(INPUT_DIM, 
              HID_DIM, 
              ENC_LAYERS, 
              ENC_HEADS, 
              ENC_PF_DIM, 
              ENC_DROPOUT, 
              device)


dec = Decoder(OUTPUT_DIM, 
              HID_DIM, 
              DEC_LAYERS, 
              DEC_HEADS, 
              DEC_PF_DIM, 
              DEC_DROPOUT,
              device)

In [7]:
SRC_PAD_IDX = STAT.vocab.stoi[STAT.pad_token]
TRG_PAD_IDX = CODE.vocab.stoi[CODE.pad_token]

model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device).to(device)

In [8]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

In [9]:
model.apply(initialize_weights)

Seq2Seq(
  (encoder): Encoder(
    (tok_embd): Embedding(6476, 256)
    (pos_embd): Embedding(10000, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttention(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc1): Linear(in_features=256, out_features=512, bias=True)
          (fc2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropou

In [10]:
LEARNING_RATE = 0.0001

In [11]:
optimizer = torch.optim.Adam(model.parameters(), lr = LEARNING_RATE)
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [12]:
import gc
# del variables
gc.collect()

264

In [13]:
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

  0%|          | 4/860 [00:00<00:27, 31.47it/s]

Epoch: 01 | Time: 0m 31s
	Train Loss: 4.343 | Train PPL:  76.907
	 Val. Loss: 3.699 |  Val. PPL:  40.405


  0%|          | 3/860 [00:00<00:32, 26.31it/s]

Epoch: 02 | Time: 0m 32s
	Train Loss: 3.456 | Train PPL:  31.688
	 Val. Loss: 3.289 |  Val. PPL:  26.814


  0%|          | 1/860 [00:00<01:39,  8.64it/s]

...

Epoch: 08 | Time: 0m 32s
	Train Loss: 1.579 | Train PPL:   4.851
	 Val. Loss: 2.083 |  Val. PPL:   8.031


  0%|          | 4/860 [00:00<00:27, 31.63it/s]

Epoch: 09 | Time: 0m 31s
	Train Loss: 1.372 | Train PPL:   3.943
	 Val. Loss: 2.016 |  Val. PPL:   7.511


100%|██████████| 860/860 [00:31<00:00, 27.02it/s]


Epoch: 10 | Time: 0m 32s
	Train Loss: 1.186 | Train PPL:   3.273
	 Val. Loss: 1.920 |  Val. PPL:   6.820


In [18]:
import spacy
# Codegen
def code_gen(sentence, src_field, trg_field, model, device, max_len = 500):

    model.eval()
        
    if isinstance(sentence, str):
        nlp = spacy.load('en')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
        
    src_indexes = [src_field.vocab.stoi[token] for token in tokens]

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)
    src_mask = model.make_src_mask(src_tensor)

    with torch.no_grad():
        enc_src = model.encoder(src_tensor,src_mask)

    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):

        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)
        trg_mask = model.make_trg_mask(trg_tensor)
        with torch.no_grad():
            output, attention = model.decoder(trg_tensor, enc_src, trg_mask, src_mask)
        
        pred_token = output.argmax(2)[:,-1].item()
        
        trg_indexes.append(pred_token)

        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break
    
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]
    
    return trg_tokens[1:]#, attention

In [15]:
trained_model = 'tut6-model.pt'
model.load_state_dict(torch.load(trained_model));
model.eval()

Seq2Seq(
  (encoder): Encoder(
    (tok_embd): Embedding(6476, 256)
    (pos_embd): Embedding(10000, 256)
    (layers): ModuleList(
      (0): EncoderLayer(
        (self_attn_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_ff_layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        (self_attention): MultiHeadAttention(
          (fc_q): Linear(in_features=256, out_features=256, bias=True)
          (fc_k): Linear(in_features=256, out_features=256, bias=True)
          (fc_v): Linear(in_features=256, out_features=256, bias=True)
          (fc_o): Linear(in_features=256, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (positionwise_feedforward): PositionwiseFeedforwardLayer(
          (fc1): Linear(in_features=256, out_features=512, bias=True)
          (fc2): Linear(in_features=512, out_features=256, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropou

In [19]:
sentence = "write a program to find and print the largest among three numbers"
code = code_gen(sentence, STAT, CODE, model, device)
#output_words[:] = [x for x in code if not (x == '<eos>' or x == '<pad>' or x == '<unk>' or x == 'utf-8')]
print(f'predicted trg = {code}')

predicted trg = ['Write', 'a', 'python', 'program', 'that', 'adds', 'a', 'list', 'of', 'square', 'python', 'element', '<eos>']


In [ ]:
translation, attention = translate_sentence(src, TEXT, TEXT, model, device)